In [11]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


import numpy as np
import pandas as pd
import string
import re
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from project_tools import project_utils, project_config

from importlib import reload
import time
import datetime
import json
import csv
from tqdm import tqdm_notebook

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1600)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
appdata = pd.read_csv('../input/20215_AppData.csv')
param = pd.read_csv('../input/20215_paramCombined.csv')
iswdata = pd.read_csv('../input/ISWdata.csv')
iswmetric = pd.read_csv('../input/ISWMetrics.csv')                      

In [27]:
pd.Series(param['UserID'].unique()).value_counts()

4    1
3    1
2    1
1    1
dtype: int64

In [30]:
param.columns

Index(['SegmentId', 'Value', 'UserId', 'Cell', 'uMotifTime', 'motifHour',
       'TimeBin', 'Day', 'Month'],
      dtype='object')

In [33]:
param.drop_duplicates(subset=['UserId', 'Cell'])['Cell'].value_counts()

3    111
2    109
4     97
1     95
Name: Cell, dtype: int64

In [17]:
appdata[~pd.isnull(appdata['Q798'])].shape

(1571, 15)

In [37]:
appdata[~pd.isnull(appdata['Q598'])]

,userid,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q798,Q598,OperationTimestamp,Cell,Day,Month,TimeBin
32,26349,11,2,1,7.000,98,2,2,NaN,Away from home in a hotel,21/11/2019 07:24,2,21,11,1
228,26250,5,3,2,nan,98,1,3,NaN,Working from home,21/11/2019 10:21,2,21,11,1
247,26366,1,3,2,nan,98,3,3,NaN,Holiday,21/11/2019 10:54,1,21,11,1
283,26481,8,1,1,7.000,98,1,3,NaN,Aldi food shopping,21/11/2019 12:08,2,21,11,2
297,26399,5,3,2,nan,98,2,3,NaN,Setting of to pick some one up from the airport,21/11/2019 12:13,4,21,11,2
485,26250,6,3,1,2.000,98,2,3,NaN,Working from home,21/11/2019 14:02,2,21,11,2
502,26366,1,3,2,nan,98,3,3,NaN,Holiday,21/11/2019 14:20,1,21,11,2
506,26485,8,2,1,7.000,98,1,3,NaN,Town centre shopping,21/11/2019 14:22,2,21,11,2
618,26349,1,3,2,nan,98,2,98,Dark,On my way home after 3 days away from home on a training course,21/11/2019 16:40,2,21,11,3
684,26512,5,3,2,nan,98,2,98,Very cold,Christmas shopping,21/11/2019 17:06,1,21,11,3


In [12]:
appdata[~pd.isnull(appdata['Q798'])]

,userid,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q798,Q598,OperationTimestamp,Cell,Day,Month,TimeBin
2,26252,1,2,2,nan,2,2,98,Cold,NaN,21/11/2019 07:02,3,21,11,1
3,26380,13,3,1,1.000,1,2,98,Dark and dull,NaN,21/11/2019 07:02,3,21,11,1
5,26354,6,3,2,nan,1,1,98,dark and cold,NaN,21/11/2019 07:04,3,21,11,1
10,26225,5,2,2,nan,1,3,98,Cold frost,NaN,21/11/2019 07:07,1,21,11,1
11,26503,1,3,2,nan,2,2,98,Cold,NaN,21/11/2019 07:07,3,21,11,1
16,26376,13,2,1,3.000,1,2,98,Frosty,NaN,21/11/2019 07:11,3,21,11,1
26,26314,1,2,2,nan,2,3,98,Very cold and dark,NaN,21/11/2019 07:19,2,21,11,1
37,26415,5,2,2,nan,2,3,98,Cold and frosty,NaN,21/11/2019 07:28,2,21,11,1
49,26248,13,2,1,3.000,1,4,98,Dull cold,NaN,21/11/2019 07:39,1,21,11,1
54,26290,5,3,2,nan,1,1,98,Frosty,NaN,21/11/2019 07:42,3,21,11,1


In [7]:
appdata.shape

(27135, 15)

In [5]:
appdata[~pd.isnull(appdata['Q598'])].shape

(463, 15)

In [6]:
appdata[~pd.isnull(appdata['Q798'])].shape

(1571, 15)

In [23]:
appdata[~pd.isnull(appdata['Q798'])]['userid'].nunique() #.sample(10)

260

In [8]:
print(iswmetric.shape)
iswmetric.head(5)

(52, 10)


,DeviceSN0x,Duration,PowerWatts,count,nodays,stdevPuffdays,nosessions,avgpuffses,puffsesstdev,UserNo
0,303847163334313828001200,1.946,10.050,201,7,28.235,10,18.000,16.062,26738.000
1,303847163334313836001500,2.470,14.651,789,20,27.869,139,5.647,7.518,26197.000
2,30384716333431383c003a00,4.102,15.120,581,25,24.241,95,6.105,5.437,26712.000
3,30384716333431383f003e00,1.719,20.000,334,9,38.303,23,14.087,10.352,26398.000
4,303847163334313840005700,2.304,10.242,207,13,14.221,26,7.885,8.199,26546.000


In [11]:
iswmetric['DeviceSN0x'].nunique()

52

In [7]:
print(iswdata.shape)
iswdata.head(5)

(142646, 6)


,DeviceSN0x,Counter,TimeStamp,Duration,PowerLevel,PowerWatts
0,3138470d3935363454002900,1.000,1970-01-01 00:00:08,1.200,High,20
1,3138470d3935363454002900,2.000,1970-01-01 00:00:12,1.500,High,20
2,3138470d3935363454002900,1.000,1970-01-01 00:20:07,3.300,Medium,15
3,3138470d3935363454002900,2.000,1970-01-01 00:20:11,3.300,High,20
4,3138470d3935363454002900,3.000,1970-01-01 00:20:16,3.300,Low,10


In [3]:
iswmetric['DeviceSN0x'].nunique()

52

In [10]:
iswdata['DeviceSN0x'].nunique()

100

In [6]:
print(param.shape)
param.head(5)

(220917, 9)


,SegmentId,Value,UserId,Cell,uMotifTime,motifHour,TimeBin,Day,Month
0,718,4,26495,3,21/11/2019 00:17,0,4,21,11
1,291,2,26350,1,21/11/2019 02:19,2,5,21,11
2,292,2,26350,1,21/11/2019 02:19,2,5,21,11
3,293,2,26350,1,21/11/2019 02:19,2,5,21,11
4,294,3,26350,1,21/11/2019 02:19,2,5,21,11


In [12]:
param['UserId'].nunique()

412

In [5]:
print(appdata.shape)
appdata.head(5)

(27135, 15)


,userid,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q798,Q598,OperationTimestamp,Cell,Day,Month,TimeBin
0,26470,5,2,1,3.000,1,2,3,NaN,NaN,21/11/2019 07:01,2,21,11,1
1,26312,2,2,2,nan,2,5,3,NaN,NaN,21/11/2019 07:01,3,21,11,1
2,26252,1,2,2,nan,2,2,98,Cold,NaN,21/11/2019 07:02,3,21,11,1
3,26380,13,3,1,1.000,1,2,98,Dark and dull,NaN,21/11/2019 07:02,3,21,11,1
4,26330,1,2,2,nan,2,3,3,NaN,NaN,21/11/2019 07:02,3,21,11,1


In [13]:
appdata['userid'].nunique()

411